In [80]:
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import os
import os.path as osp
from pypcd import pypcd
from pandas import DataFrame
import pandas as pd

In [81]:
link_to_osdar_folder = '/home/nicolasstillhard/osdar_folder'

# Create a list with all the paths contained in the link_to_osdar_folder
folders = [osp.join(link_to_osdar_folder, f) for f in os.listdir(link_to_osdar_folder) if osp.isdir(osp.join(link_to_osdar_folder, f))]

In [103]:
lidar_number_to_sensor = {
    0: 'Hesai Mid-Range',
    1: 'Livox Long-Range (1)',
    2: 'Livox Long-Range (2)',
    3: 'Livox Long-Range (3)',
    4: 'WAYMO Short-Range (1)',
    5: 'WAYMO Short-Range (2)',
}

lidar_number_to_color = {
    0: 'red',
    1: 'navy',
    2: 'cornflowerblue',
    3: 'royalblue',
    4: 'limegreen',
    5: 'forestgreen',
}

# Translate the colors to rgb values

lidar_number_to_rgb = {
    0: (1, 0, 0),
    1: (0, 0, 1),
    2: (0.39, 0.58, 0.93),
    3: (0.25, 0.41, 0.88),
    4: (0, 0.5, 0),
    5: (0.13, 0.55, 0.13),
}

### Scenes dict
Every entry in the dict contains one scene as a pandas dataframe.

In [83]:
pcd_scenes_dict = {}
all_pcds_frames = []

for folder in folders:
    # pcd files in subfolder lidar
    pcd_files = [osp.join(folder, 'lidar', f) for f in os.listdir(osp.join(folder, 'lidar')) if f.endswith('.pcd')]

    all_pcds_frames.extend(pcd_files)

    # Concatinate all numpy arrays of all the pointclouds in the folder
    pcds = []
    for pcd_file in pcd_files:

        pcd = pypcd.PointCloud.from_path(pcd_file)
        pcds.append(pcd.pc_data)

    pcds = np.concatenate(pcds)

    # Create a dataframe with the pointclouds
    df = DataFrame(pcds, columns=['x', 'y', 'z', 'intensity', 'sensor_index'])

    pcd_scenes_dict[folder] = df

### Distance Distributions
Evaluation on how the points are distributes 

In [84]:
sensor_wise_dict = {}
for scene_folder, scene_dataframe in pcd_scenes_dict.items():

    for sensor_index in scene_dataframe['sensor_index'].unique():
        if sensor_index not in sensor_wise_dict:
            sensor_wise_dict[sensor_index] = {
                'point_sampels': []
            }
        
        # Append 10% of the points of the scene to the sensor wise dict
        sensor_wise_dict[sensor_index]['point_sampels'].append(scene_dataframe[scene_dataframe['sensor_index'] == sensor_index].sample(frac=0.1))


# Concatinate all the point samples of the sensors
sensor_wise_dict_concat = {}
for sensor_index, sensor_dict in sensor_wise_dict.items():
    sensor_wise_dict_concat[sensor_index] = pd.concat(sensor_dict['point_sampels'])


In [ ]:
# Create a plot with three subplots (one for each dimesnsion)
# Every plot should show the histogram of the points of the sensor

fig, axs = plt.subplots(3, 1, figsize=(10, 10))

for i, (sensor_index, sensor_df) in enumerate(sensor_wise_dict_concat.items()):
    # Calculate the mean and variance of the points
    mean_x = sensor_df['x'].mean()
    mean_y = sensor_df['y'].mean()
    mean_z = sensor_df['z'].mean()

    var_x = sensor_df['x'].var()
    var_y = sensor_df['y'].var()
    var_z = sensor_df['z'].var()

    max_x = sensor_df['x'].max()
    min_x = sensor_df['x'].min()

    max_y = sensor_df['y'].max()
    min_y = sensor_df['y'].min()

    max_z = sensor_df['z'].max()
    min_z = sensor_df['z'].min()

    axs[0].hist(sensor_df['x'], bins=100, alpha=0.5, label=f'{lidar_number_to_sensor[sensor_index]}, mean: {mean_x:.1f}, var: {var_x:.1f}, min: {min_x:.1f}, max: {max_x:.1f}', color=lidar_number_to_color[sensor_index])
    axs[1].hist(sensor_df['y'], bins=100, alpha=0.5, label=f'{lidar_number_to_sensor[sensor_index]}, mean: {mean_y:.1f}, var: {var_y:.1f}, min: {min_y:.1f}, max: {max_y:.1f}', color=lidar_number_to_color[sensor_index])
    axs[2].hist(sensor_df['z'], bins=100, alpha=0.5, label=f'{lidar_number_to_sensor[sensor_index]}, mean: {mean_z:.1f}, var: {var_z:.1f}, min: {min_z:.1f}, max: {max_z:.1f}', color=lidar_number_to_color[sensor_index])

axs[0].legend()
axs[1].legend()
axs[2].legend()

# Set limit to the x axis of the plot on the X axis
axs[0].set_xlim(-100, 400)
axs[1].set_xlim(-40, 40)
axs[2].set_xlim(-5, 10)

axs[0].set_title('X')
axs[1].set_title('Y')
axs[2].set_title('Z')

# X-axis label
axs[2].set_xlabel('Distance [m]')
# Y-axis label
axs[1].set_ylabel('Number of points')

# Add a title to the plot
plt.suptitle('Per Sensor Total Point Cloud Distribution for 12 Scenes in OSDaR23')

plt.show()

### Statistics on the size of the pointcloud
Evaluate, how large the pointcloud for the single sensors are.

In [86]:
pc_sizes = {}

for scene_folder, scene_dataframe in pcd_scenes_dict.items():

    for sensor_index in scene_dataframe['sensor_index'].unique():
        if sensor_index not in pc_sizes:
            pc_sizes[sensor_index] = []

        pc_sizes[sensor_index].append(len(scene_dataframe[scene_dataframe['sensor_index'] == sensor_index]))

In [ ]:
# Create a histogram of the pointcloud sizes

fig, ax = plt.subplots(1, 1, figsize=(10, 10))

for sensor_index, sizes in pc_sizes.items():

    mean = np.mean(sizes)
    var = np.var(sizes)
    min_size = np.min(sizes)
    max_size = np.max(sizes)

    ax.hist(sizes, bins=50, alpha=0.5, label=f'{lidar_number_to_sensor[sensor_index]}, mean: {mean:.2f}, var: {var:.2f}, min: {min_size:.2f}, max: {max_size:.2f}', color=lidar_number_to_color[sensor_index])  
    # ax.hist(sizes, bins=50, alpha=0.5, label=f'{sensor_index}, mean: {mean:.2f}, var: {var:.2f}, min: {min_size:.2f}, max: {max_size:.2f}')

ax.legend()
ax.set_title('Pointcloud sizes')
plt.show()
    

### Investigating effects of pointcloud range

In [88]:
pointcloud_range = [0, -39.68, -3, 101.12, 39.68, 10]

In [ ]:
# Calculate the percentage, and absolute number of points that are inside the range

for scene_folder, scene_dataframe in pcd_scenes_dict.items():

    points_inside_range = scene_dataframe[(scene_dataframe['x'] >= pointcloud_range[0]) & (scene_dataframe['x'] <= pointcloud_range[3]) & (scene_dataframe['y'] >= pointcloud_range[1]) & (scene_dataframe['y'] <= pointcloud_range[4]) & (scene_dataframe['z'] >= pointcloud_range[2]) & (scene_dataframe['z'] <= pointcloud_range[5])]

    percentage_inside = len(points_inside_range) / len(scene_dataframe) * 100
    absolute_number_inside = len(points_inside_range)

    print(f'{scene_folder}: {percentage_inside:.2f}% of the points are inside the range, which is {absolute_number_inside} points')


In [ ]:
# Create a bar plot with one bar for every scene
# Each bar should show the absolute number of points that are inside the range for each individual sensor_index

fig, ax = plt.subplots(1, 1, figsize=(10, 10))

sensors = list(pcd_scenes_dict[folders[0]]['sensor_index'].unique())
sensor_labels = [lidar_number_to_sensor[sensor] for sensor in sensors]

# label shoud contain everything in the foldername after the last /
labels = [f.split('/')[-1] for f in folders]

bottom = [0] * len(labels)

for sensor in sensors:

    points_inside_range_per_sensor_percentage = []

    for scene_folder, scene_dataframe in pcd_scenes_dict.items():
            
            points_inside_range = scene_dataframe[(scene_dataframe['x'] >= pointcloud_range[0]) & (scene_dataframe['x'] <= pointcloud_range[3]) & (scene_dataframe['y'] >= pointcloud_range[1]) & (scene_dataframe['y'] <= pointcloud_range[4]) & (scene_dataframe['z'] >= pointcloud_range[2]) & (scene_dataframe['z'] <= pointcloud_range[5]) & (scene_dataframe['sensor_index'] == sensor)]

            # points inside range with sensor index == sensor
            points_inside_range_per_sensor = points_inside_range[points_inside_range['sensor_index'] == sensor]

            points_inside_range_per_sensor_percentage.append(len(points_inside_range_per_sensor)/len(scene_dataframe))

    p = ax.bar(labels, points_inside_range_per_sensor_percentage, label=f'{lidar_number_to_sensor[sensor]}', bottom=bottom, color=lidar_number_to_color[sensor])

    bottom += np.array(points_inside_range_per_sensor_percentage)
    
ax.legend()
ax.set_title('Sensor affiliation of points inside the range')

ax.set_ylabel('Percentage of points inside the range')
ax.set_xlabel('Scene')

# Rotate the x labels
plt.xticks(rotation=80)

plt.show()


### Visualize pointclouds

In [ ]:
# Chose a random pointcloud and visulize it
# Set the color of the points according to the sensor index, there is 6 different sensors

# First, load a random  pointcloud
random_pcd = np.random.choice(all_pcds_frames)

print(f'Visualizing pointcloud: {random_pcd}')

pcd = pypcd.PointCloud.from_path(random_pcd)
pcd = pcd.pc_data

# Create a numpy array with the colors of the points
colors = np.array([lidar_number_to_rgb[sensor_index] for sensor_index in pcd['sensor_index']])

# Create and visualize the pointcloud
open3d_pcd = o3d.io.read_point_cloud(random_pcd)

# Select the points that are in the range of the pointcloud
points = np.array(open3d_pcd.points)

in_range_criterion = np.logical_and.reduce([points[:, 0] > pointcloud_range[0], points[:, 0] < pointcloud_range[3], points[:, 1] > pointcloud_range[1], points[:, 1] < pointcloud_range[4], points[:, 2] > pointcloud_range[2], points[:, 2] < pointcloud_range[5]])

# Turn all points out of range grey

colors[~in_range_criterion] = (0.5, 0.5, 0.5)

open3d_pcd.colors = o3d.utility.Vector3dVector(colors)

o3d.visualization.draw_geometries([open3d_pcd])

